## Creating a Regressor for the plane class

In [1]:
# imports
from dataset import SHREC2022Primitives
from networks import MinkowskiFCNN
import transforms as t
from losses import Losses
import MinkowskiEngine as ME


import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [2]:
# configuration & hyperparameters
path = "/home/ioannis/Desktop/programming/data/SHREC/SHREC2022/dataset"
batch_size = 128
lr=1e-3
num_epochs = 20
eval_step = 1000000000

In [3]:
def minkowski_collate(list_data):
    coordinates, features, labels = ME.utils.sparse_collate(
        [d['x'] for d in list_data],
        [d['x'] for d in list_data],
        [d['y'].unsqueeze(0) for d in list_data],
        dtype = torch.float32
    )
    
    # collating other data
    norm_factors = []
    shifts = []
    inv_rotations = []
    
    for d in list_data:
        norm_factors.append(d['norm_factor'])
        shifts.append(d['shift'])
        if 'inverse_rotation' in d.keys():
            inv_rotations.append(d['inverse_rotation'])
    
    norm_factors = torch.stack(norm_factors)
    shifts = torch.stack(shifts)
    
    if len(inv_rotations) > 0:
        inv_rotations = torch.stack(inv_rotations)
    
    return {
        "coordinates"   : coordinates, 
        "features"      : features,
        "labels"        : labels,
        "norm_factors"  : norm_factors,
        "shifts"        : shifts,
        "inv_rotations" : inv_rotations if len(inv_rotations) > 0 else None
    }

In [4]:
# Dataset and dataloader
train_transforms = [t.Translate(), 
                    t.SphereNormalization(), 
                    #t.RandomRotate(180, 0),
                    #t.RandomRotate(180, 1),
                    #t.RandomRotate(180, 2),
                   ]#t.GaussianNoise()]

valid_transforms = [t.Translate(), 
                    t.SphereNormalization()]

t_dataset = SHREC2022Primitives(path, 
                                train=True, 
                                valid=False, 
                                valid_split=0.2, 
                                transform=train_transforms,
                                category="plane")

train_loader = DataLoader(t_dataset, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          collate_fn=minkowski_collate, 
                         num_workers=8)

v_dataset = SHREC2022Primitives(path, 
                                train=True, 
                                valid=True, 
                                valid_split=0.2, 
                                transform=valid_transforms,
                                category="plane")

valid_loader = DataLoader(v_dataset, 
                          batch_size=batch_size, 
                          shuffle=False, 
                          collate_fn=minkowski_collate, 
                          num_workers=8)


Specified split already exists. Using the existing one.
Specified split already exists. Using the existing one.


In [5]:
# network and device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

net = net = MinkowskiFCNN(in_channel = 3, out_channel = 6).to(device)

Using device: cuda:0


In [6]:
def create_input_batch(batch, device="cuda", quantization_size=0.05):
    batch["coordinates"][:, 1:] = batch["coordinates"][:, 1:] / quantization_size
    return ME.TensorField(
        coordinates=batch["coordinates"],
        features=batch["features"],
        device=device
    )

In [7]:
def transform_outputs(output, scale, shift, rotation_mat=None):
    
    #output: B x 9
    #scale: B x 1
    #shift: B x 3
    #rotation_mat: B x 3 x 3
    
    scale = scale.to(output.device).unsqueeze(-1)
    shift = shift.to(output.device)
    
    normal = output[:,0:3]
    vertex = output[:,3:]
    if rotation_mat is not None:
        rotation_mat = rotation_mat.to(output.device)
        #applying inverse rotation to normal vectors
        normal = (rotation_mat @ normal.unsqueeze(-1)).squeeze(-1)
    #applying inverse rotation, scaling and translation to vertices
        vertex = (rotation_mat @ vertex.unsqueeze(-1)).squeeze(-1)
    vertex = vertex * scale
    vertex = vertex - shift
    
    return normal, vertex

In [8]:
# optimizer and losses
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
criterion = Losses()

In [9]:
# Training loop

for epoch in range(num_epochs):
    # for loss and accuracy tracking the training set
    m_loss = 0
    m_vertex_loss = 0
    m_normal_loss = 0
    
    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        labels = batch["labels"].to(device)
        classes = labels[:,0].sum()

        actual_normal = labels[:,1:4]
        actual_vertex = labels[:,4:7]
         
        batch_size = labels.shape[0]

        minknet_input = create_input_batch(
            batch, 
            device=device,
            quantization_size=0.05
        )

        pred = net(minknet_input)
        pred_normal, pred_vertex = transform_outputs(pred, 
                                                     batch["norm_factors"], 
                                                     batch["shifts"], 
                                                     batch["inv_rotations"])
        

        normal_loss = criterion.AxisToAxisAngle(pred_normal, actual_normal)
        vertex_loss = criterion.PointToPlaneLoss(pred_vertex, actual_vertex, actual_normal)
        
        loss = normal_loss #+ vertex_loss
        loss.backward()
        optimizer.step()
        
        m_normal_loss += normal_loss.item()
        m_vertex_loss += vertex_loss.item()
        m_loss += loss.item()

    m_loss /= len(train_loader)
    m_normal_loss /= len(train_loader)
    m_vertex_loss /= len(train_loader)
    
    print(f" Epoch: {epoch} | Training: loss = {m_loss}")
    print(f" Normal loss: {m_normal_loss} | Vertex_loss: {m_vertex_loss}")
    
    if (epoch+1) % eval_step == 0:
        m_loss = 0
        m_normal_loss = 0
        m_vertex_loss = 0
        net.eval()
        with torch.no_grad():
            for batch in tqdm(valid_loader):

                labels = batch["labels"].to(device)
                actual_normal = labels[:,1:4]
                actual_vertex = labels[:,4:7]
                batch_size = labels.shape[0]
                
                net_in = create_input_batch(
                    batch, 
                    device=device, 
                    quantization_size=0.05
                )

                pred = net(net_in)
              
                pred_normal, pred_vertex = transform_outputs(pred, 
                                                     batch["norm_factors"], 
                                                     batch["shifts"], 
                                                     batch["inv_rotations"])

                normal_loss = criterion.AxisToAxisLoss(pred_normal, actual_normal)
                vertex_loss = criterion.PointToPlaneLoss(pred_vertex, actual_vertex, actual_normal)
        
                loss = normal_loss + vertex_loss
          
                m_normal_loss += normal_loss.item()
                m_vertex_loss += vertex_loss.item()
                m_loss += loss.item()
                
            m_loss /= len(valid_loader)
            m_normal_loss /= len(valid_loader)
            m_vertex_loss /= len(valid_loader)

            print(f" --------->  Evaluation: loss = {m_loss}")
            print(f" Normal loss: {m_normal_loss} | Vertex_loss: {m_vertex_loss}")

        # setting network back to training mode
        net.train()

print("Done!")

100%|██████████| 58/58 [00:13<00:00,  4.21it/s]


 Epoch: 0 | Training: loss = 0.013649617803507838
 Normal loss: 0.013649617803507838 | Vertex_loss: 2.3826070612874526


100%|██████████| 58/58 [00:13<00:00,  4.21it/s]


 Epoch: 1 | Training: loss = -0.01245242768320544
 Normal loss: -0.01245242768320544 | Vertex_loss: 2.357179615004309


100%|██████████| 58/58 [00:13<00:00,  4.29it/s]


 Epoch: 2 | Training: loss = -0.019952778158516718
 Normal loss: -0.019952778158516718 | Vertex_loss: 2.336129595493448


100%|██████████| 58/58 [00:13<00:00,  4.27it/s]


 Epoch: 3 | Training: loss = -0.014637743604594263
 Normal loss: -0.014637743604594263 | Vertex_loss: 3.1733704229881026


 81%|████████  | 47/58 [00:12<00:02,  3.84it/s]


KeyboardInterrupt: 